In [6]:
import psycopg2

In [7]:
from pathlib import Path

In [ ]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Create role

In [ ]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )

In [ ]:
connection

In [ ]:
crs = connection.cursor()

In [ ]:
crs.execute("SELECT rolname FROM pg_roles")
crs.fetchall()

In [ ]:
username = ""
password = ""

psql = f"""
CREATE ROLE {username}
CREATEDB 
LOGIN 
PASSWORD '{password}';"""

print(psql)
try:
    crs.execute(psql)
except Exception as e:
    print(e)

In [ ]:
crs.execute("SELECT rolname FROM pg_roles")
crs.fetchall()

In [ ]:
# COMMIT CHANGES!
connection.commit()

In [ ]:
# CLOSE CURSOR
crs.close()

In [ ]:
# CLOSE CONNECTION
connection.close()

In [ ]:
connection

# Create Database

In [ ]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [ ]:
cursor = connection.cursor()

In [ ]:
name = "garden"
user = "human"

cursor.execute(f"CREATE DATABASE {name} WITH OWNER={user}")

In [ ]:
# COMMIT CHANGES!
connection.commit()

In [ ]:
cursor.close()

In [ ]:
connection.close()

# Restart Server

In [11]:
with psycopg2.connect(
                    # the `postgres` db comes standard issue
                    database="postgres",
                    user="postgres",
                    password="DaVinci",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:

        crs = connection.cursor()

        # Find where postgres is installed (datadir and pg_ctl in bin)
        crs.execute("select name, setting from pg_settings where name = 'data_directory';")
        data_dir = Path(crs.fetchall()[0][1])
        pg_ctl = data_dir.absolute().parent / 'bin/pg_ctl'

        command = f'''cd {data_dir.absolute().parent}; sudo su postgres -c "{pg_ctl} restart -D {data_dir}"'''
        print("... restarting postgres >>>", command)
        
        crs.close()

... restarting postgres >>> cd /Library/PostgreSQL/9.5; sudo su postgres -c "/Library/PostgreSQL/9.5/bin/pg_ctl restart -D /Library/PostgreSQL/9.5/data"


# Project information

In [8]:
import psycopg2
import psycopg2.extras

In [39]:
with psycopg2.connect(
                    database="",
                    user="",
                    password="",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:
    
    sysid_columns = ['SysId','Name','LastSeen','ClientAddr','UserDefinedClientName']
    sql = f'''SELECT "{'","'.join(sysid_columns)}" FROM public."Sm2SysIdEntry" '''
    
    crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
    crs.execute(sql)
    sysids = crs.fetchall()
    
    project_columns = ['ProjectName', 'IsLiveCollaborationEnabled', 'SysIds', 'SM_Project_id']
    sql = f'''SELECT "{'","'.join(project_columns)}" FROM public."SM_Project" '''
    
    crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
    crs.execute(sql)
    projects = crs.fetchall()

In [40]:
sysids

[Record(SysId='ACBC32B39E0D', Name='Viola', LastSeen=1620088881, ClientAddr=None, UserDefinedClientName=''),
 Record(SysId='3C22FB9CB594', Name='MacBook-Pro', LastSeen=1620354931, ClientAddr='127.0.0.1', UserDefinedClientName='Jonny'),
 Record(SysId='D0817ADA2BE4', Name='Dereks-iMac-Pro', LastSeen=1620370582, ClientAddr=None, UserDefinedClientName='')]

In [41]:
projects

[Record(ProjectName='Hilbert Hotel', IsLiveCollaborationEnabled=True, SysIds='3C22FB9CB594', SM_Project_id='0ee9b114-30dc-46ba-b559-f64feba206b8')]

In [24]:
projects[0].SysIds.split(',')

['3C22FB9CB594']

In [42]:
def user_in_project(user):
    """ For a given user, what project are they in, if at all? """
    
    with psycopg2.connect(
                    # the `postgres` db comes standard issue
                    database="",
                    user="",
                    password="",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:
    
        sysid_columns = ['SysId','Name','LastSeen','ClientAddr','UserDefinedClientName']
        sql = f'''SELECT "{'","'.join(sysid_columns)}" FROM public."Sm2SysIdEntry" '''

        crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
        crs.execute(sql)
        sysids = crs.fetchall()

        project_columns = ['ProjectName', 'IsLiveCollaborationEnabled', 'SysIds', 'SM_Project_id']
        sql = f'''SELECT "{'","'.join(project_columns)}" FROM public."SM_Project" '''

        crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
        crs.execute(sql)
        projects = crs.fetchall()
    
    if len(sysids) == 0:
        # if there are no users in database this won't work...
        return
    
    user_ip = user['ip']
    
    if user['name'] == 'Server':
        # remap to home address if looking at server user
        #  because resolve reports the home address, not first_ip
        user_ip = '127.0.0.1' 

    # get sysid of user with user_ip
    for select in sysids:
        
        if user_ip == select.ClientAddr:
            # carry to after the loop
            user_sysid = select.SysId
            break
            
    else:
        print(f">>> User with ip {user_ip} was not found in Resolve database")
        return

    for project in projects:

        if project.SysIds != '':
            # Someone's in there!
            
            user_ids = project.SysIds.split(',')

            # Map to user
            
            for sysid in user_ids:
                
                if user_sysid == sysid:
                    
                    print(f">>> {user['name']} is in {project.ProjectName}")
                    
                    return project.ProjectName

In [43]:
user_in_project({'name':'Server', 'ip':'7.0.0.1'})

>>> Server is in Hilbert Hotel


'Hilbert Hotel'

In [44]:
user_in_project({'name':'Derek', 'ip':'7.0.0.7'})

>>> User with ip 7.0.0.7 was not found in Resolve database
